In [129]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException

from bs4 import BeautifulSoup
import io
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
import re
import numpy as np
import os
import glob
import shutil
import json
import html
from string import punctuation
from collections import deque

In [130]:
!pwd

/home/mark/lambda/pill_identifier


In [131]:
output_dir = "./csresults"
input_dir = "./json/"

In [132]:
class ititle_contains(object):
    """ An expectation for checking that the title contains a case-insensitive
    substring. title is the fragment of title expected
    returns True when the title matches, False otherwise
    """
    def __init__(self, title):
        self.title = title

    def __call__(self, driver):
        return self.title.lower() in html.unescape(driver.title).lower()
#         if not rv:
#             t = self.title.lower()
#             i = t.index(' mg')
#             if i > 0:
#                 t = t[0:i] + 'mg'
#                 print('t',t)
#             return t in html.unescape(driver.title).lower()


In [146]:
# todo find the picture first, then link. Most of not found were because picture was with second link and not the first
class drugscom:
    def __init__(self):
        self.debug = False
        self.first = True
        self.nonmatch_unique_file = open('nonmatch_unique.txt', 'wt')
        self.wurl = 'https://www.drugs.com/pill_identification.html'
        self.base = "https://www.drugs.com"
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        self.driver = webdriver.chrome.webdriver.WebDriver(
            '/usr/local/bin/chromedriver', options=chrome_options)
        self.ddriver = webdriver.chrome.webdriver.WebDriver(
            '/usr/local/bin/chromedriver', options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.dwait = WebDriverWait(self.ddriver, 5)
        self.driver.set_window_size(850, 1600)
        self.shapes = [
            'Round', 'Capsole', 'Oval', 'Egg', 'Barrel',
            'Rectangle', '3 Sided', '4 Sided', '5 Sided', '6 Sided',
            '7 sided', '8 sided', 'U Shaped', 'Figure 8', 'Heart',
            'Kidney', 'Gear', 'Character'
        ]
        self.shapes_classes = [
            'circle', 'capsole', 'oval', 'egg', 'barrel',
            'rectangle', 'triangle', 'square', 'pentagon', 'hexagon',
            'heptagon', 'octagon', 'U Shaped', 'Figure 8', 'Heart',
            'kidney', 'gear', 'character'
        ]
        self.colors_classes = [None, 'White', None,
                               'Beige', 'Black', 'Blue',
                               'Brown', 'Clear', 'Gold',
                               'Gray', 'Green', 'Maroon'
                               'Orange', 'Peach', 'Pink',
                               'Purple', 'Red', 'Tan',
                               'White', 'Yellow',
                               'Beige & Red', 'Black & Green',
                               'Black & Teal', 'Black & Yellow',
                               'Blue & Brown', 'Blue & Grey'
                               'Blue & Orange', 'Blue & Peach',
                               'Blue & Pink', 'Blue & White',
                               'Blue & White Specks', 'Blue & Yellow',
                               'Brown & Clear', 'Brown & Orange',
                               'Brown & Peach', 'Brown & Red',
                               'Brown & White', 'Brown & Yellow',
                               'Clear & Green', 'Dark & Light Green',
                               'Gold & White', 'Grey & Peach',
                               'Grey & Pink', 'Grey & Red',
                               'Grey & White', 'Grey & Yellow',
                               'Green & Orange', 'Green & Peach',
                               'Green & Pink', 'Green & Purple',
                               'Green & Turquoise', 'Green & White',
                               'Green & Yellow', 'Lavender & White',
                               'Maroon & Pink', 'Orange & Turquoise',
                               'Orange & White', 'Orange & Yellow',
                               'Peach & Purple', 'Peach & Red',
                               'Peach & White', 'Pink & Purple',
                               'Pink & Red Specks', 'Pink & Turquoise',
                               'Pink & White', 'Pink & Yellow',
                               'Red & Turquoise', 'Red & White',
                               'Red & Yellow', 'Tan & White',
                               'Turquoise & White', 'Turquuise & Yellow',
                               'White & Blue Specks', 'White & Red Specks',
                               'White & Yellow', 'Yellow & Grey',
                               'Yellow & White'
                               ]
        self.colors = [
            'Beige', 'Black', 'Blue',
            'Brown', 'Clear', 'Gold',
            'Gray', 'Green', 'Maroon'
            'Orange', 'Peach', 'Pink',
            'Purple', 'Red', 'Tan',
            'White', 'Yellow',
            'Beige & Red', 'Black & Green',
            'Black & Teal', 'Black & Yellow',
            'Blue & Brown', 'Blue & Grey'
            'Blue & Orange', 'Blue & Peach',
            'Blue & Pink', 'Blue & White',
            'Blue & White Specks', 'Blue & Yellow',
            'Brown & Clear', 'Brown & Orange',
            'Brown & Peach', 'Brown & Red',
            'Brown & White', 'Brown & Yellow',
            'Clear & Green', 'Dark & Light Green',
            'Gold & White', 'Grey & Peach',
            'Grey & Pink', 'Grey & Red',
            'Grey & White', 'Grey & Yellow',
            'Green & Orange', 'Green & Peach',
            'Green & Pink', 'Green & Purple',
            'Green & Turquoise', 'Green & White',
            'Green & Yellow', 'Lavender & White',
            'Maroon & Pink', 'Orange & Turquoise',
            'Orange & White', 'Orange & Yellow',
            'Peach & Purple', 'Peach & Red',
            'Peach & White', 'Pink & Purple',
            'Pink & Red Specks', 'Pink & Turquoise',
            'Pink & White', 'Pink & Yellow',
            'Red & Turquoise', 'Red & White',
            'Red & Yellow', 'Tan & White',
            'Turquoise & White', 'Turquuise & Yellow',
            'White & Blue Specks', 'White & Red Specks',
            'White & Yellow', 'Yellow & Grey',
            'Yellow & White'
        ]
        self.results = []

    def mprint_is_equal(self, m1, m2):  # m1 is drugs.com mprint, m2 is DB mprint
        m1l = m1.lower()
        m2l = m2.lower()
        if m1l == m2l:
#             print('return True', m1l)
            return True        
#         print(f'm1l|{m1l}|  m2l|{m2l}|')
        # code matching "xmg" to "x mg"
#         m1l = re.sub(r"(\s?mc?g)(?=$)", "", m1l)
#         m2l = re.sub(r"(\s?mc?g)(?=$)", "", m2l)
#         print(f'after sub m1l|{m1l}|  m2l|{m2l}|')
#         m1m = re.search(r"(mc?g)(?=$)", m1l)
#         m2m = re.search(r"(mc?g)(?=$)", m2l)
# #         m1m = re.find(r"(mg)(?=$)", m1l)
#         print('matches', m1m, m2m)
#         if (m1m != None) and (m2m != None) and (m1m.group(1) == m2m.group(1)):
#             m2l = m1l.replace(m1m.group(1), '').strip()
#             m2l = m2l.replace(m2m.group(1), '').strip()
        m1l = ''.join(c for c in m1l if c not in punctuation)
        m2l = ''.join(c for c in m2l if c not in punctuation)
        if m1l == m2l:
#             print('return True', m1l)
            return True
#         print('no match yet', m1l, m2l)
        m1s = m1l.split()
        m2s = m2l.split()
        m2sLogo = m2l.split()
        if "".join(m1s) == "".join(m2s):
#             print('returning true after dupped adjustments')
#             print('match',  "".join(m1s), "".join(m2s))
            return True
#         print('not match',  "".join(m1s), "".join(m2s))        
#         print('m1s', m1s, type(m1s), len(m1s), 'm2s', m2s, len(m2s))
        if len(m1s) == len(m2s) + 1: 
#             print('m2s befoe insert', m2s)
            m2s.insert(0, m2s[0])  # drugs.com first word dupped
            m2sLogo.insert(0, 'logo') # drugs.com added first word of 'logo'
#             print('m2s after insert', m2s)
#         print('after +1 test')
        else:
            if len(m1s) == 2 * len(m2s):  # drugs.com entire mprint dupped
                m2s.extend(m2s)
        if "".join(m1s) == "".join(m2s):
#             print('returning true after dupped adjustments')
#             print('match',  "".join(m1s), "".join(m2s))
            return True
#         print('not match',  "".join(m1s), "".join(m2s))
#         print('after dupped adjustments and no match', " ".join(m1s), " ".join(m2s))
        if len(m1s) == len(m2s):  # test every possible starting word (don't know where left/right break is)
            m2sq = deque(m2s)
            m1ss = "".join(m1l)
            for _ in range(len(m2s) - 1):
                l = m2sq.popleft()
                m2sq.append(l)
                if m1ss == "".join(m2sq):
                    return True
        return False

    def get_data(self, pmprint):
        #         self.driver.get(self.wurl)
        #         WebDriverWait(self.driver, 100).until(EC.title_contains(
        #             "Pill Identifier (Pill Finder) - Drugs.com"))
        #         try:
        #             elem = self.wait.until(
        #                 EC.element_to_be_clickable((By.LINK_TEXT, 'Accept' if len(self.results) == 0 else 'Search Again')))
        #         except:
        if self.first:
            self.driver.get(self.wurl)
            WebDriverWait(self.driver, 100).until(EC.title_contains(
                "Pill Identifier (Pill Finder) - Drugs.com"))
            if self.debug:
                print('started')
                timer.sleep(3)
                self.driver.refresh()
            else:
                elem = self.wait.until(
                    EC.element_to_be_clickable((By.LINK_TEXT, 'Accept')))
                elem.click()
            self.first = False

        mprint = pmprint
#         elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='/imprints.php']")))
        elem = self.wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "#livesearch-imprint")))
        elem.click()
        elem.clear()
        elem.send_keys(mprint)
        elem.send_keys(Keys.RETURN)
        elem = self.wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//input[@type='submit']")))
#         elem.click()
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        a = None
        mprint = None
        imgs = soup.findAll(
            lambda tag: tag.name == "img" and
            len(tag.attrs) >= 1 and
            tag["src"][0:14] == '/images/pills/')
        for img in imgs:
            #             s = img['src']
            #                       print('s',s)
            a = img.parent.parent('span', text='Pill Imprint:')[
                0].next_sibling.next_sibling
#             print('a', a, type(a), a.text)
            mprint = a.text
            # remove repeated internal spaces
            mprint = ' '.join(mprint.split())
#             print('mprint', mprint)
            if not self.mprint_is_equal(mprint, pmprint):
                #                     print('not equal',mprint,pmprint)
                if len(imgs) == 1:  # unique image result from drugs.com
                    self.nonmatch_unique_file.write(
                        f"mprint {mprint} pmprint {pmprint}")
                    break
#                 print(f"nonmatch continue mprint {mprint} pmprint {pmprint}")                    
                a = None
                mprint = None
                continue
#             print('soup breaking out of imgs loop')
            break

        try:
            elem = self.wait.until(
                EC.element_to_be_clickable((By.LINK_TEXT, 'Search Again')))
            elem.click()
        except:
            pass
        if a == None:
            return self.results
        i = 0
        if a != None:
            #             a = elem.parent.find_next('a')
            #             print(f"img: {a['href']} MPRINT: {a.text}")
            #             mprint = a.text
            #             mprint = ' '.join(mprint.split()) # remove repeated internal spaces
            #             if not self.mprint_is_equal(mprint,pmprint):
            #                 continue
            self.ddriver.get(self.base + a['href'])
            WebDriverWait(self.ddriver, 100).until(ititle_contains(mprint))
#             print(a.text, ' title')
            isoup = BeautifulSoup(self.ddriver.page_source, 'html.parser')
            div = isoup.find_all('div', {'class': 'contentBox'})[0]
            brand = div.h1.text
            brand = brand[brand.index('(') + 1:-1]
            generic = None
            try:
                generic = isoup.find_all(
                    'p', {'class': 'drug-subtitle'})[0].text
                generic = generic[14:]
            except:
                pass
            # <dt class="pid-item-title pid-item-inline">Color:</dt>
            colors = isoup.find_all('dt', string='Color:')
#             print('colors', colors)
            shapes = isoup.find_all('dt', string='Shape:')
#             imgs = isoup.find_all('img')
#             print('imgs len',len(imgs))
            appended = False
            imgs = isoup.findAll(
                lambda tag: tag.name == "img" and
                len(tag.attrs) >= 1 and
                tag["src"][0:14] == '/images/pills/')
#             print('imgs length', len(imgs))
#             print('brand', brand, 'generic', generic, mprint, )
            for img in imgs:
                #                   print('img', img)
                try:
                    s = img['src']
                    # #                       print('s',s)
                    #                     if s[0:14] == '/images/pills/':
                        #                          print('found img', s, ' mprint ', mprint)
                        #                         self.img = base + s
                    self.results.append(
                        {'brand': brand, 'generic': generic, 'mprint': mprint, 'img': self.base + s,
                         'color': colors[i].next_sibling.text, 'shape': shapes[i].next_sibling.text})
                    appended = True
                    break
                except Exception as e:
                    print('error appending', repr(e))
                    pass
#             if self.mprint_is_equal(mprint,pmprint) & appended: # works because drugs.com puts matching mprint first
#                 break
#             print( mprint.lower(), pmprint.lower())
            i += i
        return self.results

    def reset(self):
        self.results = []

    def close(self):
        self.driver.quit()
        self.ddriver.quit()
        self.nonmatch_unique_file.close()

In [151]:
d = drugscom()
test = d.get_data("Andrx 696 120 mg")
test1 = d.get_data('Westward 480')
d.get_data('M Amphet Salts 30 mg')
test = d.get_data('BAYER 20')
test = d.get_data('BAYER 10')
test = d.get_data('M T6')
test = d.get_data('2876')
d.close()

In [152]:
test

[{'brand': 'Taztia XT 120 mg',
  'generic': 'diltiazem',
  'mprint': 'Andrx 696 120mg',
  'img': 'https://www.drugs.com/images/pills/mtm/620370696_PB.jpg',
  'color': 'Pink',
  'shape': 'Capsule-shape'},
 {'brand': 'Propylthiouracil 50 mg',
  'generic': None,
  'mprint': 'West-ward 480',
  'img': 'https://www.drugs.com/images/pills/mtm/001431480_PB.jpg',
  'color': 'White',
  'shape': 'Round'},
 {'brand': 'Amphetamine and Dextroamphetamine Extended Release 30 mg',
  'generic': None,
  'mprint': 'M. Amphet Salts 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/001151333.jpg',
  'color': 'Orange / Clear',
  'shape': 'Capsule-shape'},
 {'brand': 'Levitra 20 mg',
  'generic': 'vardenafil',
  'mprint': 'BAYER BAYER 20',
  'img': 'https://www.drugs.com/images/pills/nlm/000851934.jpg',
  'color': 'Orange',
  'shape': 'Round'},
 {'brand': 'Levitra 10 mg',
  'generic': 'vardenafil',
  'mprint': 'BAYER BAYER 10',
  'img': 'https://www.drugs.com/images/pills/nlm/000851901.jpg',
  'color':

In [145]:
d.close()

In [109]:
pd.set_option('display.max_columns', 500)

In [110]:
url = 'pillbox_201805.tab'
df_ = pd.read_csv(url,sep='\t')

In [111]:
df = df_[(df_.SPLIMPRINT.notnull()) & (df_.image_id.notnull()) & (df_.image_id != 'no_product_image')]

In [112]:
df.head()

,id,spl_id,SETID,spp,NDC9,PRODUCT_CODE,EQUAL_PRODUCT_CODE,author,SPLIMPRINT,SPLCOLOR,SPLSHAPE,SPLSIZE,SPLSCORE,DEA_SCHEDULE_CODE,INGREDIENTS,SPL_INACTIVE_ING,RXCUI,RXTTY,RXSTRING,image_id,IMAGE_SOURCE,HAS_IMAGE,FROM_SIS,NO_RXCUI
50,4710,NaN,8fed703a-9de2-4c76-a783-fcf004f9b890,8fed703a-9de2-4c76-a783-fcf004f9b890-50844-175-0,508440175,50844-175,NaN,"L.N.K. International, Inc.",44;175,C48325,C48345,17.0,1,NaN,ACETAMINOPHEN[ACETAMINOPHEN],HYPROMELLOSES / POVIDONE / STEARIC ACID / CAST...,198440.0,SCD,Acetaminophen 500 MG Oral Tablet,50844-0175-10_NLMIMAGE10_5135A8AD,NLMIMAGE10,1,NaN,NaN
58,61985,NaN,c35d5dcc-8ee1-49da-993d-65c0195db918,c35d5dcc-8ee1-49da-993d-65c0195db918-70518-0901-0,705180901,70518-0901,0093-7206,REMEDYREPACK INC.,9;3;7206,C48330,C48348,7.0,2,NaN,MIRTAZAPINE[MIRTAZAPINE],"SILICON DIOXIDE / STARCH, CORN / HYPROMELLOSE ...",311725.0,SCD,Mirtazapine 15 MG Oral Tablet,00093720656,SPL,1,1.0,NaN
68,11648,NaN,2fb7763f-6ce4-43b9-b3be-90a66315828b,2fb7763f-6ce4-43b9-b3be-90a66315828b-60429-566-0,604290566,60429-566,0591-0744,"Golden State Medical Supply, Inc.",WATSON;744;1,C48325,C48338,9.0,2,C48677,ESTAZOLAM[ESTAZOLAM],DOCUSATE SODIUM / LACTOSE MONOHYDRATE / MAGNES...,NaN,NaN,NaN,604290566,VA2,1,1.0,NaN
71,47756,NaN,662f11ca-cf1e-41fd-9b2a-11a23d1f6290,662f11ca-cf1e-41fd-9b2a-11a23d1f6290-0781-1034-0,007811034,0781-1034,NaN,Sandoz Inc,GG55;5,C48327,C48348,8.0,1,NaN,TRIFLUOPERAZINE HYDROCHLORIDE[TRIFLUOPERAZINE],D&C RED NO. 30 / FD&C BLUE NO. 2 / HYDROXYPROP...,198325.0,SCD,Trifluoperazine 5 MG Oral Tablet,007811034,VA2,1,NaN,NaN
79,33477,NaN,70be7d33-bc8b-4f48-84c6-a8e3d1476e97,70be7d33-bc8b-4f48-84c6-a8e3d1476e97-59762-0145-0,597620145,59762-0145,NaN,Greenstone LLC,FELDENE;PFIZER;323,C48326,C48336,18.0,1,NaN,PIROXICAM[PIROXICAM],"FD&C BLUE NO. 1 / FD&C RED NO. 3 / LACTOSE, UN...",198108.0,SCD,Piroxicam 20 MG Oral Capsule,59762-0145-01_NLMIMAGE10_333E99F4,RXIMAGE30,1,NaN,NaN


In [113]:
df[df.SPLIMPRINT == "C;10"]

,id,spl_id,SETID,spp,NDC9,PRODUCT_CODE,EQUAL_PRODUCT_CODE,author,SPLIMPRINT,SPLCOLOR,SPLSHAPE,SPLSIZE,SPLSCORE,DEA_SCHEDULE_CODE,INGREDIENTS,SPL_INACTIVE_ING,RXCUI,RXTTY,RXSTRING,image_id,IMAGE_SOURCE,HAS_IMAGE,FROM_SIS,NO_RXCUI
19515,18599,NaN,bcd8f8ab-81a2-4891-83db-24a0b0e25895,bcd8f8ab-81a2-4891-83db-24a0b0e25895-0002-4463-0,000024463,0002-4463,NaN,Eli Lilly and Company,C;10,C48330,C48351,11.0,1,NaN,Tadalafil[Tadalafil],Hydroxypropyl cellulose (Type H) / Hypromellos...,402096.0,SBD,tadalafil 10 MG Oral Tablet [Cialis],00002-4463-30_B5155ACA,NLM,1,NaN,NaN
33345,27495,NaN,2ca7efcd-4a8d-4139-a3eb-930afd18b13f,2ca7efcd-4a8d-4139-a3eb-930afd18b13f-57237-005-0,572370005,57237-005,NaN,Citron Pharma LLC,C;10,C48328,C48336,12.0,1,NaN,FLUCONAZOLE[FLUCONAZOLE],"CELLULOSE, MICROCRYSTALLINE / ANHYDROUS DIBASI...",197699.0,SCD,Fluconazole 150 MG Oral Tablet,57237-0005-11_NLMIMAGE10_34471A28,RXIMAGE30,1,NaN,NaN
43091,35129,NaN,d2acccda-8068-46c1-8562-950d7f70ee06,d2acccda-8068-46c1-8562-950d7f70ee06-16714-692-0,167140692,16714-692,NaN,NorthStar Rx LLC,C;10,C48328,C48336,12.0,1,NaN,FLUCONAZOLE[FLUCONAZOLE],"CELLULOSE, MICROCRYSTALLINE / ANHYDROUS DIBASI...",197699.0,SCD,Fluconazole 150 MG Oral Tablet,16714-0692-10_NLMIMAGE10_10408864,RXIMAGE30,1,NaN,NaN


In [114]:
regex = r"([\w\d]+)"

In [115]:
js_names = glob.glob(os.path.join(input_dir, '*.json')) 

In [116]:
print(len(js_names))

7001


In [117]:
d = drugscom()

In [118]:
#  Andrx 696 120 mg find image in brand

In [119]:
dicta = []
i = 0
for fp in js_names:
    image_id = fp[7:-9]
    d.reset()
    j = json.load(open(fp, 'r'))
    mprint = j['description']
    mprint = ' '.join(mprint.split()) # remove repeated internal spaces
    dict = d.get_data(mprint)
    if len(dict) == 0:
        print(f"mprint {mprint} in {image_id} not found")
        i += 1
        continue
    dict = dict[0]
#     print(mprint, d.get_data(mprint), fp[9:-9])
    try: 
        row = df[df.image_id == image_id][['SPLCOLOR','SPLSHAPE']].iloc[0]
#     print('|' + image_id + '|', row['SPLCOLOR'], row['SPLSHAPE'], type(row))
        dicta.append({'image_id': image_id, 'color': dict['color'],'SPLCOLOR': row['SPLCOLOR'], 'shape': dict['shape'],'SPLSHAPE': row['SPLSHAPE']  })
    except:
        print(f"row {mprint} {image_id} not found")
    i += 1
    if i % 100 == 0:
        print('i',i,end=' ')
#     break

mprint 2876 in 009551046 not found
mprint L U X04 in 68180-0115-07_NLMIMAGE10_3A081D50 not found
mprint 10 crestor in 433530290 not found
mprint M Tofranil PM 75mg in 004069923 not found
mprint RX526 in 680840248 not found
mprint PA 309 in 50111-0309-01_8629434A not found
mprint IG321 100mg in 604290738 not found
mprint N 305 800 in 000938305 not found
mprint SB 4894 in 00007-4894-20_C718E3D7 not found
mprint TCL272 in 422910312 not found
mprint DAN DAN 5325 in 00591-5325-01_240E1210 not found


TimeoutException: Message: timeout
  (Session info: chrome=72.0.3626.119)
  (Driver info: chromedriver=2.45.615279 (12b89733300bd268cff3b78fc76cb8f3a7cc44e5),platform=Linux 4.15.0-47-generic x86_64)


In [113]:
d.close()

In [111]:
d = drugscom()

In [112]:
# d = drugscom()
time.sleep(2)
d.reset()
d.get_data("C 10")
d.get_data("LILLY 3240 30 mg")
d.get_data("LILLY 4117")

[{'brand': 'Cialis 10 mg',
  'generic': 'tadalafil',
  'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/mmx/t106434f/cialis.jpg',
  'color': 'Yellow',
  'shape': 'Elliptical / Oval'},
 {'brand': 'Cymbalta 30 mg',
  'generic': 'duloxetine',
  'mprint': 'Lilly 3240 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/000023240.jpg',
  'color': 'Blue & White',
  'shape': 'Capsule-shape'},
 {'brand': 'Zyprexa 10 mg',
  'generic': 'olanzapine',
  'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg',
  'color': 'White',
  'shape': 'Round'}]

In [113]:
d.get_data("10 c")

[{'brand': 'Fluconazole 150 mg',
  'generic': None,
  'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/mtm/572370005_PB.jpg',
  'color': 'Pink',
  'shape': 'Capsule-shape'}]

In [92]:
d.reset()
# d.get_data("LILLY 4117")

In [61]:
 d.get_data("4117 lilly")

[{'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg',
  'color': 'White',
  'shape': 'Round'},
 {'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg',
  'color': 'White',
  'shape': 'Round'}]

In [129]:
len(dicta)

2053

In [116]:
dfd = pd.DataFrame(dicta) #todo never name a DataFrame column "shape" again

In [117]:
dfd.head()

,SPLCOLOR,SPLSHAPE,color,image_id,shape
0,C48328,C48348,Pink,433530866,Round
1,C48329;C48327,C48336,Green & Purple,00093316006,Capsule-shape
2,C48325,C48348,White,510790810,Round
3,C48325,C48345,White,68180-0115-07_NLMIMAGE10_3A081D50,Elliptical / Oval
4,C48325,C48348,White,00093514056,Round


In [118]:
with open('color_shape.csv', 'wt') as f:
    dfd.to_csv(f)

In [119]:
!pwd

/home/mark/lambda/pill_identifier


In [120]:
ushapes = dfd['shape'].unique()
sdict = {}
for splshape in dfd.SPLSHAPE.unique():
    rd = {}
    for shape in ushapes:
        c = len(dfd[(dfd.SPLSHAPE == splshape) & (dfd['shape'] == shape)])
        if c > 0:
            rd[shape] = c
    sdict[splshape] = rd

In [121]:
ucolors = dfd.color.unique()
cdict = {}
for splcolor in dfd.SPLCOLOR.unique():
    rd = {}
    for color in ucolors:
        c = len(dfd[(dfd.SPLCOLOR == splcolor) & (dfd.color == color)])
        if c > 0:
            rd[color] = c
    cdict[splcolor] = rd

In [122]:
from functools import reduce

In [123]:
sd = {}
for cd in sdict.keys():
    d = sdict[cd]
    da = reduce((lambda m, v: [d[v],v] if d[v] > m[0] else m), d.keys(), [0,'x'])
    sd[cd] = da[1]

In [124]:
sd

{'C48348': 'Round',
 'C48336': 'Capsule-shape',
 'C48345': 'Elliptical / Oval',
 'C48346': 'Five-sided',
 'C48340': 'U-shape',
 'C48350': 'Four-sided',
 'C48338': 'Four-sided',
 'C48351': 'Egg-shape',
 'C48347': 'Rectangle',
 'C48353': 'Three-sided',
 'C48352': 'Four-sided',
 'C48349': 'U-shape',
 'C48343': 'Six-sided',
 'C48344': 'Eight-sided',
 'C48335': 'Barrel',
 'C48339': 'Figure eight-shape'}

In [125]:
fd = {}
for cd in cdict.keys():
    d = cdict[cd]
    da = reduce((lambda m, v: [d[v],v] if d[v] > m[0] else m), d.keys(), [0,'x'])
    fd[cd] = da[1]

In [126]:
ssd = {}
for key in sorted(sd.keys()):
    ssd[key] = sd[key]

In [127]:
ssd

{'C48335': 'Barrel',
 'C48336': 'Capsule-shape',
 'C48338': 'Four-sided',
 'C48339': 'Figure eight-shape',
 'C48340': 'U-shape',
 'C48343': 'Six-sided',
 'C48344': 'Eight-sided',
 'C48345': 'Elliptical / Oval',
 'C48346': 'Five-sided',
 'C48347': 'Rectangle',
 'C48348': 'Round',
 'C48349': 'U-shape',
 'C48350': 'Four-sided',
 'C48351': 'Egg-shape',
 'C48352': 'Four-sided',
 'C48353': 'Three-sided'}

In [128]:
sfd = {}
for key in sorted(fd.keys()):
    sfd[key] = fd[key]

In [129]:
for key in fd.keys():
    sk = key.split(';')
    sd = sfd[key].split(' & ')
    if len(sd) == 1:
        sd = sfd[key].split(' / ')
    subkeys = (len(sk) == len(sd)) & (len(sd) > 1)
    r = subkeys
    if subkeys:
        for i in range(len(sd)):
            try:
                if sfd[sk[i]] not in sd:
                    r = False
                    break
            except:
                r = False
                break;
    if r:
        if r & subkeys:
            print('deleting key ', key)
        del sfd[key]
            
            

deleting key  C48329;C48327
deleting key  C48329;C48328
deleting key  C48329;C48325
deleting key  C48325;C48333
deleting key  C48333;C48330
deleting key  C48326;C48325
deleting key  C48333;C48325
deleting key  C48325;C48331
deleting key  C48323;C48329
deleting key  C48331;C48325
deleting key  C48325;C48330
deleting key  C48333;C48329
deleting key  C48331;C48330
deleting key  C48325;C48324
deleting key  C48328;C48329
deleting key  C48333;C48328
deleting key  C48327;C48328
deleting key  C48330;C48325
deleting key  C48324;C48330
deleting key  C48325;C48329
deleting key  C48328;C48325
deleting key  C48325;C48332
deleting key  C48331;C48329
deleting key  C48330;C48326
deleting key  C48329;C48330
deleting key  C48331;C48332
deleting key  C48324;C48325
deleting key  C48323;C48330
deleting key  C48327;C48331
deleting key  C48327;C48325
deleting key  C48330;C48332
deleting key  C48326;C48333
deleting key  C48333;C48324
deleting key  C48330;C48333
deleting key  C48333;C48331
deleting key  C48329

In [130]:
for key in fd.keys():
    if key in sfd:
        if len(key.split(';')) > 1: 
            del sfd[key]

In [131]:
sfd

{'C48323': 'Black',
 'C48324': 'Gray',
 'C48325': 'White',
 'C48326': 'Red',
 'C48327': 'Purple',
 'C48328': 'Pink',
 'C48329': 'Green',
 'C48330': 'Yellow',
 'C48331': 'Orange',
 'C48332': 'Brown',
 'C48333': 'Blue',
 'C48334': 'Blue'}